
# Project 1: Build an LLM Playground

Welcome! In this project, you’ll learn foundations of large language models (LLMs). We’ll keep the code minimal and the explanations high‑level so that anyone who can run a Python cell can follow along.  

We'll be using Google Colab for this project. Colab is a free, browser-based platform that lets you run Python code and machine learning models without installing anything on your local computer. Click the button below to open this notebook directly in Google Colab and get started!


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bytebyteai/ai-eng-projects/blob/main/project_1/lm_playground.ipynb)

---
## Learning Objectives  
* **Tokenization** and how raw text is tokenized into a sequene of discrete tokens
* Inspect **GPT2** and **Transformer architecture**
* Loading pre-trained LLMs using **Hugging Face**
* **Decoding strategies** to generate text from LLMs
* Completion versus **intrusction fine-tuned** LLMs


Let's get started!

In [1]:
import torch, transformers, tiktoken
print("torch", torch.__version__, "| transformers", transformers.__version__)

torch 2.8.0+cu126 | transformers 4.57.1


# 1 - Tokenization

A neural network can’t digest raw text. They need **numbers**. Tokenization is the process of converting text into IDs. In this section, you'll learn how tokenization is implemented in practice.

Tokenization methods generally fall into three categories:
1. Word-level
2. Character-level
3. Subword-level

### 1.1 - Word‑level tokenization

Split text on whitespace and store each **word** as a token.

In [2]:
# 1. Tiny corpus
corpus = [
    "The quick brown fox jumps over the lazy dog",
    "Tokenization converts text to numbers",
    "Large language models predict the next token"
]

# 2. Build the vocabulary
PAD, UNK = "[PAD]", "[UNK]"
vocab = [PAD, UNK]
unique_words = set(word for sent in corpus for word in sent.split())
vocab.extend(sorted(unique_words))

word2id = {w: i for i, w in enumerate(vocab)}
id2word = {i: w for w, i in word2id.items()}

print(f"Vocabulary size: {len(vocab)} words")
print("First 15 vocab entries:", vocab[:15])

# 3. Encode / decode
def encode(text):
    tokens = text.split()
    return [word2id.get(w, word2id[UNK]) for w in tokens]

def decode(ids):
    return " ".join(id2word.get(i, UNK) for i in ids)

# 4. Demo
sample = "The brown unicorn jumps"
ids = encode(sample)
recovered = decode(ids)

print("\nInput text :", sample)
print("Token IDs  :", ids)
print("Decoded    :", recovered)


Vocabulary size: 22 words
First 15 vocab entries: ['[PAD]', '[UNK]', 'Large', 'The', 'Tokenization', 'brown', 'converts', 'dog', 'fox', 'jumps', 'language', 'lazy', 'models', 'next', 'numbers']

Input text : The brown unicorn jumps
Token IDs  : [3, 5, 1, 9]
Decoded    : The brown [UNK] jumps


Word-level tokenization has two major limitations:
1. Large vocabulary size
2. Out-of-vocabulary (OOV) issue

### 1.2 - Character‑level tokenization

Every single character (including spaces and emojis) gets its own ID. This guarantees zero out‑of‑vocabulary issues but very long sequences.

In [3]:
# 1. Build a fixed vocabulary: a–z + A–Z + padding + unknown
import string

PAD, UNK = "[PAD]", "[UNK]"
vocab = [PAD, UNK] + list(string.ascii_lowercase + string.ascii_uppercase)
char2id = {c: i for i, c in enumerate(vocab)}
id2char = {i: c for c, i in char2id.items()}

print(f"Vocabulary size: {len(vocab)} (52 letters + 2 specials)")

# 2. Encode / decode
def encode(text):
    return [char2id.get(ch, char2id[UNK]) for ch in text]

def decode(ids):
    return "".join(id2char.get(i, UNK) for i in ids)

# 3. Demo
sample = "Hello"
ids = encode(sample)
recovered = decode(ids)
print("\nInput text :", sample)
print("Token IDs  :", ids)
print("Decoded    :", recovered)


Vocabulary size: 54 (52 letters + 2 specials)

Input text : Hello
Token IDs  : [35, 6, 13, 13, 16]
Decoded    : Hello


### 1.3 - Subword‑level tokenization

Sub-word methods such as `Byte-Pair Encoding (BPE)`, `WordPiece`, and `SentencePiece` **learn** the most common character and gorup them into new tokens. For example, the word `unbelievable` might turn into three tokens: `["un", "believ", "able"]`. This approach strikes a balance between word-level and character-level methods and fix their limitations.

For example, `BPE` algorithm forms the vocabulary using the following steps:
1. **Start with bytes** → every character is its own token.  
2. **Count all adjacent pairs** in a huge corpus.  
3. **Merge the most frequent pair** into a new token.  
   *Repeat steps 2-3* until you hit the target vocab size (e.g., 50 k).

Let's see `BPE` in practice.

In [4]:
# 1. Load a pretrained BPE tokenizer (GPT-2 uses BPE)
from transformers import AutoTokenizer

bpe_tok = AutoTokenizer.from_pretrained("gpt2")

# 2. Encode / decode
def encode(text):
    return bpe_tok.encode(text, add_special_tokens=False)

def decode(ids):
    return bpe_tok.decode(ids)

# 3. Demo
sample = "Unbelievable tokenization powers! 🚀"
ids = encode(sample)
recovered = decode(ids)

print("\nInput text :", sample)
print("Token IDs  :", ids)
print("Tokens     :", bpe_tok.convert_ids_to_tokens(ids))
print("Decoded    :", recovered)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]


Input text : Unbelievable tokenization powers! 🚀
Token IDs  : [3118, 6667, 11203, 540, 11241, 1634, 5635, 0, 12520, 248, 222]
Tokens     : ['Un', 'bel', 'iev', 'able', 'Ġtoken', 'ization', 'Ġpowers', '!', 'ĠðŁ', 'ļ', 'Ģ']
Decoded    : Unbelievable tokenization powers! 🚀


### 1.4 - TikToken

`tiktoken` is a production-ready library which offers high‑speed tokenization used by OpenAI models.  
Let's compare the older **gpt2** encoding with the newer **cl100k_base** used in GPT‑4.

In [5]:
# Use gpt2 and cl100k_base to encode and decode the following text
import tiktoken

sentence = "The 🌟 star-player scored 40 points!"

enc_gpt2 = tiktoken.get_encoding("gpt2")
enc_cl100k = tiktoken.get_encoding("cl100k_base")

ids_gpt2 = enc_gpt2.encode(sentence)
ids_cl100k = enc_cl100k.encode(sentence)

print("GPT-2 encoding IDs :", ids_gpt2)
print("GPT-2 decoded      :", enc_gpt2.decode(ids_gpt2))

print("\ncl100k_base IDs    :", ids_cl100k)
print("cl100k_base decoded:", enc_cl100k.decode(ids_cl100k))


GPT-2 encoding IDs : [464, 12520, 234, 253, 3491, 12, 7829, 7781, 2319, 2173, 0]
GPT-2 decoded      : The 🌟 star-player scored 40 points!

cl100k_base IDs    : [791, 11410, 234, 253, 6917, 43467, 16957, 220, 1272, 3585, 0]
cl100k_base decoded: The 🌟 star-player scored 40 points!


Experiment: try new sentences, emojis, code snippets, or other languages. If you are interested, try implementing the BPE algorithm yourself.

### 1.5 - Key Takeaways

* **Word‑level**: simple but brittle (OOV problems).  
* **Character‑level**: robust but produces long sequences.  
* **BPE / Byte‑Level BPE**: middle ground used by most LLMs.  
* **tiktoken**: shows how production models tokenize with pre‑trained sub‑word vocabularies.

# 2. What is a Language Model?

At its core, a **language model (LM)** is just a *very large* mathematical function built from many neural-network layers.  
Given a sequence of tokens `[t₁, t₂, …, tₙ]`, it learns to output a probability for the next token `tₙ₊₁`.


Each layer applies a simple operation (matrix multiplication, attention, etc.). Stacking hundreds of these layers lets the model capture patterns and statistical relations from text. The final output is a vector of scores that says, “how likely is each possible token to come next?”

> Think of the whole network as **one gigantic equation** whose parameters were tuned during training to minimize prediction error.



### 2.1 - A Single `Linear` Layer

Before we explore Transformer, let’s start tiny:

* A **Linear layer** performs `y = Wx + b`  
  * `x` – input vector  
  * `W` – weight matrix (learned)  
  * `b` – bias vector (learned)

Although this looks basic, chaining thousands of such linear transforms (with nonlinearities in between) gives neural nets their expressive power.


In [8]:
import torch
import torch.nn as nn

class Linear(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        # initialize weights and bias manually to show what's happening
        self.W = nn.Parameter(torch.randn(out_features, in_features) * 0.01)
        self.b = nn.Parameter(torch.zeros(out_features))

    def forward(self, x):
        # y = Wx + b
        return x @ self.W.T + self.b

# test
lin = Linear(3, 2)
x = torch.tensor([1.0, -1.0, 0.5])
print("Input :", x)
print("Weights:\n", lin.W)
print("Bias   :", lin.b)
print("Output :", lin(x))


Input : tensor([ 1.0000, -1.0000,  0.5000])
Weights:
 Parameter containing:
tensor([[-0.0009, -0.0078, -0.0118],
        [ 0.0090, -0.0048, -0.0111]], requires_grad=True)
Bias   : Parameter containing:
tensor([0., 0.], requires_grad=True)
Output : tensor([0.0010, 0.0083], grad_fn=<AddBackward0>)


In [9]:
import torch.nn as nn, torch

lin = nn.Linear(3, 2)
x = torch.tensor([1.0, -1.0, 0.5])
print("Input :", x)
print("Weights:", lin.weight)
print("Bias   :", lin.bias)
print("Output :", lin(x))


Input : tensor([ 1.0000, -1.0000,  0.5000])
Weights: Parameter containing:
tensor([[ 0.5726,  0.4862, -0.4981],
        [ 0.0594,  0.0534, -0.3092]], requires_grad=True)
Bias   : Parameter containing:
tensor([-0.1530,  0.1401], requires_grad=True)
Output : tensor([-0.3157, -0.0084], grad_fn=<ViewBackward0>)


### 2.2 - A `Transformer` Layer

Most LLMs are a **stack of identical Transformer blocks**. Each block fuses two main components:

| Step | What it does | Where it lives in code |
|------|--------------|------------------------|
| **Multi-Head Self-Attention** | Every token looks at every other token and decides *what matters*. | `block.attn` |
| **Feed-Forward Network (MLP)** | Re-mixes information token-by-token. | `block.mlp` |

Below, we load the smallest public GPT-2 (124 M parameters), grab its *first* block, and inspect the pieces.


In [10]:
import torch
from transformers import GPT2LMHeadModel

# Load smallest GPT-2
gpt2 = GPT2LMHeadModel.from_pretrained("gpt2")

# Inspect one Transformer block
block = gpt2.transformer.h[0]
print(block)


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2Block(
  (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (attn): GPT2Attention(
    (c_attn): Conv1D(nf=2304, nx=768)
    (c_proj): Conv1D(nf=768, nx=768)
    (attn_dropout): Dropout(p=0.1, inplace=False)
    (resid_dropout): Dropout(p=0.1, inplace=False)
  )
  (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (mlp): GPT2MLP(
    (c_fc): Conv1D(nf=3072, nx=768)
    (c_proj): Conv1D(nf=768, nx=3072)
    (act): NewGELUActivation()
    (dropout): Dropout(p=0.1, inplace=False)
  )
)


In [11]:
# Run a tiny forward pass through the first block
seq_len = 8
dummy_tokens = torch.randint(0, gpt2.config.vocab_size, (1, seq_len))

with torch.no_grad():
    # Embed tokens + positions
    x = gpt2.transformer.wte(dummy_tokens) + gpt2.transformer.wpe(torch.arange(seq_len))
    # Forward through one layer
    out = block(x)

print("Output shape:", out[0].shape)  # (batch, seq_len, hidden_size)


Output shape: torch.Size([1, 8, 768])


### 2.3 - Inside GPT-2

GPT-2 is just many of those modules arranged in a repeating *block*. Let's print the modules inside the Transformer.

In [12]:
# Print the names and modules inside GPT-2
for name, module in gpt2.named_children():
    print(f"{name}: {module.__class__.__name__}")


transformer: GPT2Model
lm_head: Linear


As you can see, the Transformer holds various modules, arranged from a list of blocks (`h`). The following table summarizes these modules:

| Step | What it does | Why it matters |
|------|--------------|----------------|
| **Token → Embedding** | Converts IDs to vectors | Gives the model a numeric “handle” on words |
| **Positional Encoding** | Adds “where am I?” info | Order matters in language |
| **Multi-Head Self-Attention** | Each token asks “which other tokens should I look at?” | Lets the model relate words across a sentence |
| **Feed-Forward Network** | Two stacked Linear layers with a non-linearity | Mixes information and adds depth |
| **LayerNorm & Residual** | Stabilize training and help gradients flow | Keeps very deep networks trainable |


### 2.4 LLM's output

Passing a token sequence through an **LLM** yields a tensor of **logits** with shape  
`(batch_size, seq_len, vocab_size)`.  
Applying `softmax` on the last dimension turns those logits into probabilities.

The cell below feeds an 8-token dummy sequence, prints the logits shape, and shows the five most likely next tokens for the final position.


In [13]:
import torch
import torch.nn.functional as F
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

# Load GPT-2 model and tokenizer
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Tokenize input
text = "Hello my name"
inputs = tokenizer(text, return_tensors="pt")

# Get logits
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

print("Logits shape:", logits.shape)  # (batch, seq_len, vocab_size)

# Predict next token
next_token_logits = logits[0, -1, :]
probs = F.softmax(next_token_logits, dim=-1)
topk = torch.topk(probs, 5)

print("\nTop-5 predictions for the next token:")
for idx, score in zip(topk.indices, topk.values):
    print(f"{tokenizer.decode(idx)} : {score.item():.4f}")


Logits shape: torch.Size([1, 3, 50257])

Top-5 predictions for the next token:
 is : 0.7773
, : 0.0373
's : 0.0332
 was : 0.0127
 and : 0.0076


### 2.5 - Key Takeaway

A language model is nothing mystical: it’s a *huge composition* of small, understandable layers trained to predict the next token in a sequence of tokens.

# 3 - Generation
Once an LLM is trained to predict the probabilities, we can generate text from the model. This process is called decoding or sampling.

At each step, the LLM outputs a **probability distribution** over the next token. It is the job of the decoding algorithm to pick the next token, and move on to the next token. There are different decoding algorithms and hyper-parameters to control the generaiton:
* **Greedy** → pick the single highest‑probability token each step (safe but repetitive).  
* **Top‑k / Nucleus (top‑p)** → sample from a subset of likely tokens (adds variety).
* **beam** -> applies beam search to pick tokens
* **Temperature** → a *creativity* knob. Higher values flatten the probability distribution.

### 3.1 - Greedy decoding

In [14]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Available models
MODELS = {
    "gpt2": "gpt2",
}

# Load and cache models + tokenizers
tokenizers, models = {}, {}
for key, name in MODELS.items():
    tok = AutoTokenizer.from_pretrained(name)
    mdl = AutoModelForCausalLM.from_pretrained(name)
    tok.pad_token = tok.eos_token
    models[key] = mdl
    tokenizers[key] = tok


def generate(model_key, prompt, strategy="greedy", max_new_tokens=100, temperature=1.0, top_k=50, top_p=0.9):
    """Generate text using GPT-2 with different decoding strategies."""
    tok, mdl = tokenizers[model_key], models[model_key]
    inputs = tok(prompt, return_tensors="pt")

    if strategy == "greedy":
        # always pick the highest probability token each step
        output = mdl.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            temperature=temperature,
            pad_token_id=tok.eos_token_id,
        )

    elif strategy == "top-k":
        output = mdl.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_k=top_k,
            temperature=temperature,
            pad_token_id=tok.eos_token_id,
        )

    elif strategy in ["top-p", "nucleus"]:
        output = mdl.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_p=top_p,
            temperature=temperature,
            pad_token_id=tok.eos_token_id,
        )

    else:
        raise ValueError(f"Unknown strategy: {strategy}")

    return tok.decode(output[0], skip_special_tokens=True)


In [15]:
tests = ["Once upon a time", "What is 2+2?", "Suggest a party theme."]
for prompt in tests:
    print(f"\n== GPT-2 | Greedy ==")
    print(generate("gpt2", prompt, strategy="greedy", max_new_tokens=80))



== GPT-2 | Greedy ==
Once upon a time, the world was a place of great beauty and great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and

== GPT-2 | Greedy ==
What is 2+2?

2+2 is the number of times you can use a spell to cast a spell.

2+2 is the number of times you can use a spell to cast a spell.

2+2 is the number of times you can use a spell to cast a spell.

2+2 is the number of times you can use a spell to cast a spell.

== GPT-2 | Greedy ==
Suggest a party theme.

The party theme is a simple, simple, and fun way to get your friends to join you.

The party theme is a simple, simple, and fun way to get your friends to join you. The party theme is a simple, simple, and fun way to get your friends to join you. The party theme is a simple


Naively picking the single best token every time has the following issues in practice:

* **Loop**: “The cat is is is…”  
* **Miss long-term payoff**: the highest-probability word *now* might paint you into a boring corner later.

### 3.2 - Top-k or top-p sampling

In [16]:
tests = ["Once upon a time", "What is 2+2?", "Suggest a party theme."]
for prompt in tests:
    print(f"\n== GPT-2 | Top-p Sampling ==")
    print(generate("gpt2", prompt, strategy="top-p", max_new_tokens=60))



== GPT-2 | Top-p Sampling ==
Once upon a time in my life, my friend and I decided to build up a little bit of a relationship that I could share with my friends and family. In particular, I was interested in a book by a book reviewer, but had been turned down by her review because it was so uninspired, which was why

== GPT-2 | Top-p Sampling ==
What is 2+2?


The only way a number is valid is if it is 2. For example:


1. 1+2 = 2

2. 2+1 = 1+2 = 2+2 = 2+1 = 2+2 = 2+1 = 2+2 = 2

== GPT-2 | Top-p Sampling ==
Suggest a party theme.

Here's what the current game looks like for our new theme:

The base game is about a group of refugees who've been held hostage by a group of government-sponsored terrorists on a border in Central America. In order to save the group, they must be rescued by a military


### 3.3 - Try It Yourself

1. Scroll to the list called `tests`.
2. Swap in your own prompts or tweak the decoding strategy.  
3. Re‑run the cell and compare the vibes.

> **Tip:** Try the same prompt with `greedy` vs. `top_p` (0.9) and see how the tone changes. Notice especially how small temperature tweaks can soften or sharpen the prose.

* `strategy`: `"greedy"`, `"beam"`, `"top_k"`, `"top_p"`  
* `temperature`: `0.2 – 2.0`  
* `k` or `p` thresholds



# 4 - Completion vs. Instruction-tuned LLMs

We have seen that we can use GPT2 model to pass an input text and generate a new text. However, this model only continues the provided text. It is not engaging in a dialouge-like conversation and cannot be helpful by answering instructions. On the other hand, **instruction-tuned LLMs** like `Qwen-Chat` go through an extra training stage called **post-training** after the base “completion” model is finished. Because of post-training step, an instruction-tuned LLM will:

* **Read the entire prompt as a request,** not just as text to mimic.  
* **Stay in dialogue mode**. Answer questions, follow steps, ask clarifying queries.  
* **Refuse or safe-complete** when instructions are unsafe or disallowed.  
* **Adopt a consistent persona** (e.g., “Assistant”) rather than drifting into story continuation.


### 4.1 - Qwen1.5-8B vs. GPT2

In the code below we’ll feed the same prompt to:

* **GPT-2 (completion-only)** – it will simply keep writing in the same style.  
* **Qwen-Chat (instruction-tuned)** – it will obey the instruction and respond directly.

Comparing the two outputs makes the difference easy to see.

In [17]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Define models to compare
MODELS = {
    "gpt2": "gpt2",
    "qwen": "Qwen/Qwen1.5-1.8B-Chat"
}

tokenizers, models = {}, {}

# Load models and tokenizers
for key, name in MODELS.items():
    print(f"Loading {key} ...")
    tok = AutoTokenizer.from_pretrained(name)
    mdl = AutoModelForCausalLM.from_pretrained(name)
    tok.pad_token = tok.eos_token if tok.pad_token is None else tok.pad_token
    tokenizers[key] = tok
    models[key] = mdl
print("✅ Models and tokenizers loaded.")


Loading gpt2 ...
Loading qwen ...


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

✅ Models and tokenizers loaded.



We downloaded two tiny checkpoints: `GPT‑2` (124 M parameters) and `Qwen‑1.5‑Chat` (1.8 B). If the cell took a while, that was mostly network time. Models are stored locally after the first run.

Let's now generate text and compare two models.


In [18]:
tests = [
    ("Once upon a time", "greedy"),
    ("What is 2+2?", "top_k"),
    ("Suggest a party theme.", "top_p")
]

for prompt, strategy in tests:
    for key in ["gpt2", "qwen"]:
        print(f"\n== {key.upper()} | {strategy} ==")
        print(generate(key, prompt, strategy, 80))



== GPT2 | greedy ==


The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Once upon a time, the world was a place of great beauty and great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and

== QWEN | greedy ==
Once upon a time, there was a young girl named Lily who lived in a small village nestled among the rolling hills of her home. Lily had always been fascinated by nature and spent most of her days exploring the surrounding forests, streams, and meadows. She loved to sing songs about the birds that sang in the trees, the flowers that bloomed in the fields, and the animals that roamed free.
One day

== GPT2 | top_k ==


ValueError: Unknown strategy: top_k

# 5. (Optional) A Small LLM Playground

### 5.1 ‑ Interactive Playground

Enter a prompt, pick a model and decoding strategy, adjust the temperature, and press **Generate** to watch the model respond.


In [19]:
import ipywidgets as widgets
from IPython.display import display, Markdown

# Ensure models and tokenizers are loaded from previous cells
try:
    tokenizers
    models
except NameError:
    raise RuntimeError("Please run the earlier setup cells that load the models before using the playground.")

# Function to generate text from the playground UI
def generate_playground(model_key, prompt, strategy="greedy", temperature=1.0, max_new_tokens=100, top_k=50, top_p=0.9):
    """Interactive text generation wrapper using the global `generate()` function."""
    if not prompt.strip():
        return "⚠️ Please enter a prompt first."
    try:
        output = generate(
            model_key=model_key,
            prompt=prompt,
            strategy=strategy,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            max_new_tokens=max_new_tokens
        )
        return output
    except Exception as e:
        return f"❌ Error: {str(e)}"

# -----------------------------
# 🧩 Build UI Elements
# -----------------------------
prompt_box = widgets.Textarea(
    value="Tell me a joke about AI and coffee.",
    placeholder="Enter your prompt here...",
    description="Prompt:",
    layout=widgets.Layout(width="100%", height="80px")
)

model_dropdown = widgets.Dropdown(
    options=list(models.keys()),
    value="gpt2",
    description="Model:"
)

strategy_dropdown = widgets.Dropdown(
    options=["greedy", "top-k", "top-p"],
    value="top-p",
    description="Strategy:"
)

temperature_slider = widgets.FloatSlider(
    value=1.0,
    min=0.2,
    max=2.0,
    step=0.1,
    description="Temperature:",
    continuous_update=False
)

max_tokens_slider = widgets.IntSlider(
    value=80,
    min=10,
    max=300,
    step=10,
    description="Max tokens:",
    continuous_update=False
)

topk_slider = widgets.IntSlider(
    value=50,
    min=1,
    max=200,
    step=5,
    description="Top-k:",
    continuous_update=False
)

topp_slider = widgets.FloatSlider(
    value=0.9,
    min=0.1,
    max=1.0,
    step=0.05,
    description="Top-p:",
    continuous_update=False
)

generate_button = widgets.Button(
    description="🚀 Generate",
    button_style="success",
    tooltip="Generate text using the selected model and settings"
)

output_area = widgets.Output()

# -----------------------------
# ⚙️ Define Interaction Logic
# -----------------------------
def on_generate_click(_):
    output_area.clear_output()
    with output_area:
        display(Markdown(f"### **Prompt:**\n> {prompt_box.value.strip()}"))
        print("\n---\n")
        print("Generating... please wait ⏳\n")
        text = generate_playground(
            model_key=model_dropdown.value,
            prompt=prompt_box.value,
            strategy=strategy_dropdown.value,
            temperature=temperature_slider.value,
            max_new_tokens=max_tokens_slider.value,
            top_k=topk_slider.value,
            top_p=topp_slider.value
        )
        print(text)

generate_button.on_click(on_generate_click)

# -----------------------------
# 🎛️ Layout the UI
# -----------------------------
ui = widgets.VBox([
    widgets.HBox([model_dropdown, strategy_dropdown]),
    widgets.HBox([temperature_slider, max_tokens_slider]),
    widgets.HBox([topk_slider, topp_slider]),
    prompt_box,
    generate_button,
    output_area
])

display(ui)



## 🎉 Congratulations!

You’ve just learned, explored, and inspected a real **LLM**. In one project you:
* Learned how **tokenization** works in practice
* Used `tiktoken` library to load and experiment with most advanced tokenizers.
* Explored LLM architecture and inspected GPT2 blocks and layers
* Learned decoding strategies and used `top-p` to generate text from GPT2
* Loaded a powerful chat model, `Qwen1.5-8B` and generated text
* Built an LLM playground


👏 **Great job!** Take a moment to celebrate. You now have a working mental model of how LLMs work. The skills you used here power most LLMs you see everywhere.
